# नमूना 01: OpenAI SDK के माध्यम से त्वरित चैट

यह नोटबुक Microsoft Foundry Local के साथ OpenAI SDK का उपयोग करके त्वरित चैट इंटरैक्शन कैसे करें, इसका प्रदर्शन करती है।

## अवलोकन

यह उदाहरण दिखाता है:
- Foundry Local के साथ OpenAI Python SDK का उपयोग करना
- Azure OpenAI और स्थानीय Foundry कॉन्फ़िगरेशन दोनों को संभालना
- उचित त्रुटि प्रबंधन और बैकअप रणनीतियों को लागू करना
- सेवा प्रबंधन के लिए FoundryLocalManager का उपयोग करना


## आवश्यकताएँ

सुनिश्चित करें कि आपके पास निम्नलिखित हैं:
1. Foundry Local इंस्टॉल और चालू है
2. एक मॉडल लोड किया हुआ है (जैसे, `foundry model run phi-4-mini`)
3. आवश्यक Python पैकेज इंस्टॉल किए हुए हैं

### डिपेंडेंसी इंस्टॉल करें


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## आवश्यक लाइब्रेरी आयात करें


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## कॉन्फ़िगरेशन

अपनी कॉन्फ़िगरेशन सेट करें। आप निम्न में से किसी एक का उपयोग कर सकते हैं:
1. **Azure OpenAI** (क्लाउड-आधारित)
2. **Foundry Local** (स्थानीय अनुमान)

### विकल्प 1: Azure OpenAI कॉन्फ़िगरेशन


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### विकल्प 2: फाउंड्री स्थानीय कॉन्फ़िगरेशन


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## क्लाइंट प्रारंभिककरण

अपनी कॉन्फ़िगरेशन के आधार पर OpenAI क्लाइंट बनाएं:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## सरल चैट उदाहरण

आइए एक साधारण चैट इंटरैक्शन का परीक्षण करें:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## इंटरएक्टिव चैट सत्र

मॉडल की प्रतिक्रिया देखने के लिए विभिन्न प्रकार के प्रॉम्प्ट आज़माएं:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## उन्नत उपयोग: स्ट्रीमिंग प्रतिक्रियाएँ

लंबी प्रतिक्रियाओं के लिए, स्ट्रीमिंग बेहतर उपयोगकर्ता अनुभव प्रदान कर सकती है:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## त्रुटि प्रबंधन और निदान

आइए सेवा की स्थिति और उपलब्ध मॉडलों की जांच करें:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## कस्टम प्रॉम्प्ट परीक्षण

नीचे दिए गए सेल का उपयोग करके अपने प्रॉम्प्ट का परीक्षण करें:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## सारांश

इस नोटबुक में निम्नलिखित का प्रदर्शन किया गया:

1. **✅ क्लाइंट सेटअप**: Foundry Local के साथ OpenAI SDK को कॉन्फ़िगर करने का तरीका
2. **✅ बेसिक चैट**: सरल अनुरोध-प्रतिक्रिया इंटरैक्शन
3. **✅ स्ट्रीमिंग**: वास्तविक समय में प्रतिक्रिया उत्पन्न करना
4. **✅ त्रुटि प्रबंधन**: मजबूत त्रुटि प्रबंधन और डायग्नोस्टिक्स
5. **✅ सेवा स्वास्थ्य**: मॉडल की उपलब्धता और सेवा की स्थिति की जांच करना

### अगले कदम

- **सैंपल 02**: Azure OpenAI समर्थन के साथ उन्नत SDK एकीकरण
- **सैंपल 04**: Chainlit चैट एप्लिकेशन बनाना
- **सैंपल 05**: मल्टी-एजेंट ऑर्केस्ट्रेशन सिस्टम
- **सैंपल 06**: बुद्धिमान मॉडल रूटिंग

### Foundry Local के मुख्य लाभ

- 🔒 **गोपनीयता**: डेटा आपके डिवाइस से बाहर नहीं जाता
- ⚡ **गति**: कम विलंबता स्थानीय अनुमान
- 💰 **लागत**: कोई API उपयोग लागत नहीं
- 🔌 **ऑफ़लाइन**: इंटरनेट कनेक्शन के बिना काम करता है
- 🛠️ **संगतता**: OpenAI-संगत API
